In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
from transformers import AutoProcessor, AutoModelForImageClassification
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
df_allCarabids = pd.read_csv("/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/AllCarabids.csv", sep="\t")
df_insect1M = df_allCarabids[df_allCarabids["filepath"].str.startswith("Insect-1M")].copy()
df_insect1M.rename(columns={"filepath": "ImageFilePath", "class": "ScientificName"}, inplace=True)
cols = ['ImageFilePath', 'ScientificName']
df = df_insect1M[cols]
df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + df['ImageFilePath']
df = df.dropna()
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "microsoft/swinv2-base-patch4-window8-256"

processor = AutoProcessor.from_pretrained(model_name, use_fast=True)

model = AutoModelForImageClassification.from_pretrained(model_name).to(device).eval()


In [5]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        features = outputs.logits
    
    return features.cpu().numpy()


In [6]:

X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing ...")])

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)



Processing ...:   0%|          | 0/20278 [00:00<?, ?it/s]

In [7]:
test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:

seed = 45

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

metrics = {}

predictions = {}

for name, model in tqdm(models.items(), desc="Training Models", leave=True):

    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")

metrics_df = pd.DataFrame(metrics).T


Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 9.66% | Prec: 12.43% | Rec: 9.66% | F1: 8.64% | Bal Acc: 7.62% | MCC: 0.0952
LogisticRegression        | Acc: 42.78% | Prec: 44.37% | Rec: 42.78% | F1: 41.03% | Bal Acc: 27.15% | MCC: 0.4259
NearestNeighbor           | Acc: 19.80% | Prec: 19.52% | Rec: 19.80% | F1: 17.49% | Bal Acc: 9.57% | MCC: 0.1952
MLP_Baseline              | Acc: 38.61% | Prec: 38.84% | Rec: 38.61% | F1: 36.75% | Bal Acc: 24.08% | MCC: 0.3842


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})

In [10]:
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
8222,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela willistoni,Habroscelimorpha dorsalis,Ellipsoptera nevadica,Habroscelimorpha dorsalis,Habroscelimorpha dorsalis
15309,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela hirticollis,Cicindela pulchra,Cicindela hirticollis,Cicindela hirticollis,Cicindela hirticollis


In [11]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.096647,0.124335,0.096647,0.08643,0.076223,0.095218
LogisticRegression,LogisticRegression,0.427761,0.443686,0.427761,0.410341,0.271458,0.425899
NearestNeighbor,NearestNeighbor,0.197978,0.195216,0.197978,0.174871,0.095697,0.195189
MLP_Baseline,MLP_Baseline,0.386095,0.388403,0.386095,0.367475,0.240794,0.384163


In [12]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Insect1M_dropna/5.SWINv2-Insect1M_dropna-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Insect1M_dropna/5.SWINv2-Insect1M_dropna-species-metrics.csv", index=False)